# 8장. 조인

## 8.1 > JOIN 타입

- inner join
- outer join
    - left outer join
    - right outer join
- left semi join
- left anti join
- natural join
- cross join

In [2]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
.toDF("id", "status")

person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

## 8.2 > 내부조인 INNER JOIN : 교집합

In [8]:
joinType = "inner"

joinExpression = person["graduate_program"] == graduateProgram["id"]


# table1.join(table2, 조건, 조인 타입(default = "inner"))
person.join(graduateProgram, joinExpression, joinType).show()

# SQL
# SELECT * FROM person JOIN graduateProgram ON person.graduate_program = graduateProgram.id

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



## 8.3 > 외부조인 OUTER JOIN : 합집합

In [11]:
# outer join은 해당사항이 없으면 null 값으로 채움
joinType = "outer"

# table1.join(table2, 조건, 조인 타입(default = "inner"))
person.join(graduateProgram, joinExpression, joinType).show()

# SQL
# SELECT * FROM person FULL OUTER JOIN graduateProgram ON person.graduate_program = graduateProgram.id

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



#### 왼쪽 외부 조인  left outer join : A, B를 나타내지만 A만 표시

In [14]:
# left join은 왼쪽 table에 맞춰서 생성
joinType = "left_outer"

# A.join(B, 조건, 조인 타입(default = "inner"))
person.join(graduateProgram, joinExpression, joinType).show()
graduateProgram.join(person, joinExpression, joinType).show()

# SQL
# SELECT * FROM person LEFT OUTER JOIN graduateProgram ON person.graduate_program = graduateProgram.id

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---

#### 오른쪽 외부 조인  right outer join : A, B를 나타내지만 B만 표시

In [15]:
# right join은 오른쪽 table에 맞춰서 생성
joinType = "right_outer"

# A.join(B, 조건, 조인 타입(default = "inner"))
person.join(graduateProgram, joinExpression, joinType).show()
graduateProgram.join(person, joinExpression, joinType).show()

# SQL
# SELECT * FROM person RIGHT OUTER JOIN graduateProgram ON person.graduate_program = graduateProgram.id

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_progra

## 8.4 > 왼쪽 세미 조인 LEFT SEMI JOIN
- 기존 조인과 달리 필터기능을 함 (A만 보여줌)
- 오른쪽 DataFrame(B)의 어떤 정보도 보여주지 않고 중복되는 값만 왼쪽 DataFrame(A)에서 보여줌
- A와 B의 교집합 중 A의 데이터만 보여줌

In [23]:
joinType = "left_semi"

# A.join(B 조건, 조인 타입(default = "inner"))
graduateProgram.join(person, joinExpression, joinType).show()
graduateProgram.show()

# SQL
# SELECT * FROM graduateProgram LEFT SEMI JOIN person ON person.graduate_program = graduateProgram.id

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Ph.D|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|   Ph.D|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



## 8.5 > 왼쪽 안티 조인 LEFT ANTI JOIN
- 기존 조인과 달리 필터기능을 함
- 세미조인의 반대 : 중복되지 않는 데이터를 보여줌
- (A-B) 중 A의 데이터만 보여줌

In [24]:
joinType = "left_anti"

# table1.join(table2, 조건, 조인 타입(default = "inner"))
graduateProgram.join(person, joinExpression, joinType).show()
graduateProgram.show()

# SQL
# SELECT * FROM graduateProgram FULL OUTER JOIN person ON person.graduate_program = graduateProgram.id

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|   Ph.D|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



## 8.6 > 자연조인 NATURAL JOIN
- 조인 조건을 주지 않고 암시적으로 컬럼을 찾아서 조인을 실행
- 왼쪽, 오른쪽, 외부 자연조인을 사용할 수 있음
- !!! 암시적인 데이터 처리이기에 매우 위험

In [25]:
# SQL
# SELECT * FROM graduateProgram NATURAL JOIN person

## 8.7 > 교차조인 CROSS JOIN
- cross 조인은 모든 경우를 row로 표현

In [30]:
joinType = "cross"

# table1.join(table2, 조건, 조인 타입(default = "inner"))
graduateProgram.join(person, joinExpression, joinType).show()

person.crossJoin(graduateProgram).show()

# SQL
# SELECT * FROM graduateProgram CROSS JOIN person ON person.graduate_program = graduateProgram.id
# SELECT * FROM graduateProgram CROSS JOIN person

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|   Ph.D|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|   Ph.D|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+--

## 8.8 > 조인++

#### 1. 복합데이터 타입의 조인
- array_contains 를 표현식으로 활용

In [32]:
from pyspark.sql.functions import expr

# withColumnRenamed : 컬럼 이름 변경
person.withColumnRenamed("id", "personId")\
.join(sparkStatus, expr("array_contains(spark_status, id)")).show()

# SQL
# SELECT * FROM 
#     (SELECT id as personId, naem, graduate_program, spark_status FROM person)
#         INNER JOIN sparkStatus ON array_contains(spark_status, id)

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



#### 2. 중복 컬럼명 처리
- SQL에서는 정확히 명시가 가능 
- But, DataFrame의 각 컬럼은 Spark SQL엔진인 카탈리스트 내에 고유 ID가 존재 > 직접 참조가 불가...'
<br>
- 해결방법 
    - 다른 조인 표현식 사용
    - 조인 후 컬럼 제거
    - 조인 전 컬럼명 변경
    
##### 하지만 pyspark는 그런거없는듯~~

In [38]:
from pyspark.sql.functions import col

gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")

joinExpression = gradProgramDupe["graduate_program"] == person["graduate_program"]

person.join(gradProgramDupe, joinExpression).show()

+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|graduate_program| degree|          department|     school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|               0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|               1|   Ph.D|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|               1|   Ph.D|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+



## 8.9 > 스파크의 조인 수행 방식

#### 1. 노드간 네트워크 통신 전략
- 셔플 조인 (전체 노드 간 통신)
    - 큰 테이블과 큰 테이블의 조인
        - 전체 조인 프로세스가 진행되는 동안 모든 워커 노드에서 통신이 발생
<br><br>
- 브로드캐스트 조인 (일부 노드 간 통신)
    - 큰 테이블과 작은 테이블 조인 (테이블이 단일 워커 노드의 메모리 크기에 적합할 정도로 충분히 작은 경우)
        - DataFrame을 클러스터의 전체 워커 노드에 복제
        - 전체 통신을 방지 : 시작시 한번만 복제가 일어남
        - 단일 노드에서 개별적으로 조인이 수행 >> CPU가 가장 큰 병목구간이 됨
        - 너무 큰 데이터를 브로드캐스트 하면 고비용의 수집연산이 발생하기에 드라이버 노드가 비정상적 종료
        <br><br>
        - DataFrame API를 이용하면 옵티마이저에 브로드캐스트 조인을 하도록 유도 가능 (강제성 없음)
            - SCALA : person.join(broadcast(graduateProgram), joinExpr).explain()
            - SQL : MAPJOIN, BROADCAST, BROADCASTJOIN 등 힌트 가능
<br><br>    
- 아주작은 테이블 사이의 조인은 내버려두면 spark가 알아서 잘함ㅋㅋ
    

## ++
- 적절한 데이터 분할로 같은 머신에 두 DF가 존재하면 조인 연산에 있어서 효율이 향상될 수도
- 일부데이터 실험용으로 빼서 확인하고 데이터의 적절한 분할이 중요